**Clean Data**

In [4]:
import pandas as pd


df = pd.read_csv('carData.csv')
df.loc[df['fuel_type'].isnull() & df['engine'].str.contains('Electric Motor', case=False, na=False), 'fuel_type'] = 'Electric'

df.loc[df['fuel_type'].isnull()].head()
df.loc[df['fuel_type'].isnull() & (df['engine'].str.lower() == 'electric'), 'fuel_type'] = 'Electric'
df.loc[df['fuel_type'].isnull() & (df['brand'].str.lower() == 'tesla'), 'fuel_type'] = 'Electric'
df.loc[df['fuel_type'].isnull() & df['engine'].str.contains('battery', case=False, na=False), 'fuel_type'] = 'Electric'

df['milage'] = df['milage'].str.replace(r'[^\d]', '', regex=True).astype(int)
df['price'] = df['price'].replace(r'[$,]', '', regex=True).astype(int)

df['fuel_type'] = df['fuel_type'].replace('Plug-In Hybrid', 'Hybrid')
df['clean_title'] = df['clean_title'].fillna('No')
df['accident'] = df['accident'].fillna('Unknown')
df.to_csv('cleaned_car_data.csv',index=False)

df = pd.read_csv('cleaned_car_data.csv')
df['model'].unique().shape[0]


# df.head()

1898

In [24]:
import pandas as pd
import plotly.express as px


df = pd.read_csv('cleaned_car_data.csv')
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data
df = pd.read_csv('cleaned_car_data.csv')

# Create histogram bins
counts, bins = np.histogram(df['price'], bins=50)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# Create bar chart with text labels
fig = go.Figure(data=[
    go.Bar(
        x=bin_centers,
        y=counts,
        text=counts,
        textposition='outside',
        name='Car Count'
    )
])

# Add vertical line at $250,000
fig.add_vline(
    x=250000,
    line_color='red',
    line_dash='dash',
    line_width=2,
    annotation_text='Outlier Threshold ($250k)',
    annotation_position='top right'
)

# Update layout for log scale and tick labels
fig.update_layout(
    title='Car Price Distribution',
    xaxis_title='Price ($)',
    yaxis_title='Count (Log Scale)',
    yaxis_type='log',
    yaxis=dict(
        tickvals=[1, 10, 100, 1000, 10000],
        ticktext=['1', '10', '100', '1k', '10k']
    )
)

fig.show()




# print(df['price'].shape[0])
# print(df[df['price']>250000].shape[0])


In [29]:
df = pd.read_csv('cleaned_car_data.csv')

# Create price category
df['Price Category'] = df['price'].apply(lambda x: '≤ $250k' if x <= 250000 else '> $250k')

# Count category occurrences
price_counts = df['Price Category'].value_counts().reset_index()
price_counts.columns = ['Category', 'Count']

fig = px.pie(
    price_counts,
    names='Category',
    values='Count',
    title='Proportion of Cars by Price Category',
    color='Category',
    color_discrete_map={
        '≤ $250k': 'lightblue',
        '> $250k': 'red'
    }
)

# Bigger label and percent text
fig.update_traces(
    textinfo='label+percent+value',
    textfont_size=15  # Increase this for larger text
)

# Bigger title and layout
fig.update_layout(
    title_font_size=24,
    width=600,
    height=600
)

fig.show()


In [6]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv('cleaned_car_data.csv')

df.nunique()

df.head()



def extract_hp(engine_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*hp', str(engine_str).lower())  # handles float like 211.0
    if match:
        return float(match.group(1))
    return np.nan

df['HP'] = df['engine'].apply(extract_hp)

df['HP'] = df['engine'].apply(extract_hp)

df['HP'].head()
df.to_csv('test.csv')


**Split Data**

In [19]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 1. Load the dataset
df = pd.read_csv('cleaned_car_data.csv')
df = df[df['price'] < 250000]  # Remove extreme outliers

# 2. Convert 'clean_title' to binary
df['clean_title'] = df['clean_title'].map({'Yes': 1, 'No': 0}).astype(int)

# 3. Target encode high-cardinality text features
df['brand_encoded'] = df.groupby('brand')['price'].transform('mean')
df['model_encoded'] = df.groupby('model')['price'].transform('mean')
df['engine_encoded'] = df.groupby('engine')['price'].transform('mean')
df['transmission_encoded'] = df.groupby('transmission')['price'].transform('mean')

# 4. Extract HP from engine string
def extract_hp(engine_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*hp', str(engine_str).lower())
    if match:
        return float(match.group(1))
    return np.nan

df['HP'] = df['engine'].apply(extract_hp)
df['HP'].fillna(df['HP'].median(), inplace=True)  # Fill missing HP values with median

# 5. Bin 'milage' to reduce noise
df['milage_binned'] = pd.qcut(df['milage'], q=10, labels=False)

# 6. Define feature groups
numerical_features = ['model_year', 'milage_binned', 'HP']  # Added HP here
binary_features = ['clean_title']
target_encoded_features = ['brand_encoded', 'model_encoded', 'engine_encoded', 'transmission_encoded']
categorical_features = ['fuel_type', 'ext_col', 'int_col', 'accident']

# 7. One-hot encode remaining low-cardinality categorical features
df_encoded = pd.get_dummies(df[categorical_features], drop_first=True)

# 8. Combine features into X
X = pd.concat([df[numerical_features + binary_features + target_encoded_features], df_encoded], axis=1)
y = df['price']

# 9. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 10. Train RandomForestRegressor
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=0,
    n_jobs=-1
)
model.fit(X_train, y_train)

# 11. Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {np.sqrt(mse):.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R^2 Score: {r2:.4f}")

# 12. Feature importances
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance_df.to_csv('feature_importance.csv', index=False)

# 13. Save predictions with errors
X_test_copy = X_test.copy()
X_test_copy['predicted_price'] = y_pred
X_test_copy['actual_price'] = y_test.values
X_test_copy['pct_error'] = ((X_test_copy['predicted_price'] - X_test_copy['actual_price']) / X_test_copy['actual_price']) * 100
X_test_copy['abs_pct_error'] = X_test_copy['pct_error'].abs()
X_test_copy.to_csv('test_predictions_with_errors.csv', index=False)

# Root Mean Squared Error: 16370.49
# Mean Absolute Error: 5440.66
# R^2 Score: 0.8656


C:\Users\amans\AppData\Local\Temp\ipykernel_2960\1592262394.py:29: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Root Mean Squared Error: 9207.52
Mean Absolute Error: 4792.89
R^2 Score: 0.9308


In [17]:
import plotly.express as px
import pandas as pd
# Create a DataFrame for comparison
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred,
    'Brand': df.loc[y_test.index, 'brand']
})


fig = px.scatter(
    results_df,
    x='Actual Price',
    y='Predicted Price',
    title='Actual vs Predicted Car Prices',
    labels={'Actual Price': 'Actual Price ($)', 'Predicted Price': 'Predicted Price ($)'},
    opacity=0.6,
    trendline="ols",
    hover_name='Brand'
)


fig.add_shape(
    type="line",
    x0=results_df['Actual Price'].min(), y0=results_df['Actual Price'].min(),
    x1=results_df['Actual Price'].max(), y1=results_df['Actual Price'].max(),
    line=dict(color="red", dash="dash"),
    name='Perfect Prediction'
)


fig.show()


In [18]:
import plotly.express as px

# Ensure brand column exists in X_test_copy
X_test_copy['brand'] = df.loc[X_test_copy.index, 'brand']

# 1. Compute percentage error
X_test_copy['pct_error'] = ((X_test_copy['predicted_price'] - X_test_copy['actual_price']) / X_test_copy['actual_price']) * 100

# 2. Compute average percent error per brand
avg_pct_error_df = X_test_copy.groupby('brand', as_index=False)['pct_error'].mean()
avg_pct_error_df.rename(columns={'pct_error': 'Avg % Error'}, inplace=True)

# 3. Sort by average percent error
avg_pct_error_df = avg_pct_error_df.sort_values(by='Avg % Error')

# 4. Plot
fig = px.bar(
    avg_pct_error_df,
    x='brand',
    y='Avg % Error',
    color='brand',
    title='Average Percent Error by Brand',
    labels={'Avg % Error': 'Average % Error', 'brand': 'Brand'},
)
fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()
